In [1]:
import csv, json, os, re, shutil
import subprocess, sys, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [3]:
psql('\d', database='census2010')

\d
Finished execution in 2.43191 secs: List of relations
 Schema |            Name             |   Type   |   Owner    
--------+-----------------------------+----------+------------
 public | geo2000                     | table    | census2010
 public | geo2010                     | table    | census2010
 public | geography_columns           | view     | postgres
 public | geometry_columns            | view     | postgres
 public | raster_columns              | view     | postgres
 public | raster_overviews            | view     | postgres
 public | sf1_01                      | table    | census2010
 public | sf1_02                      | table    | census2010
 public | sf1_03                      | table    | census2010
 public | sf1_04                      | table    | census2010
 public | sf1_05                      | table    | census2010
 public | sf1_06                      | table    | census2010
 public | sf1_07                      | table    | census2010
 public | sf1_08   

Data seems to be here:
    
http://www2.census.gov/census_2010/

    including demographic profile, summary files 1 and 2
    
    
    http://www2.census.gov/census_2010/03-Demographic_Profile/0README_DPSF.pdf:
    
The data in the 2010 Census Demographic Profile Summary File and other 2010 Census summary
files are segmented. This is done so that individual files will not have more than 255 fields,
facilitating exporting into spreadsheet or database software. In short, to get a complete set of the
2010 Census Demographic Profile Summary File, users must download both files in the
package.

The files are as follows:
    Geographic Header Record file
    File01 (DPSF1-DPSF23)
    
It is easiest to think of the file set as a logical file. This logical file consists of two physical files:
the geographic header file and file01. This structure is the same as the 2010 Census
Redistricting Data files. 

Summary File 1
--------------

Summary File 1 is described in http://www.census.gov/prod/cen2010/doc/sf1.pdf

Urban, rural, race, sex, incl sex by age.  Can use SF1 to get ages 18-64 if we want



Looks like Daniel et al imported 2010 Census Summary File 1 to earthserve2 in census2010 database

census2010 database uses "logrecno" as the common join key.  Not all tables actually use logrecno as an index however, beware.

census2010=# select sum(p012001) from sf1_04 inner join geo2010 ON sf1_04.logrecno = geo2010.logrecno WHERE geo2010.sumlev='101';



Working age population
----------------------

St Louis Fed defines as ages 15-64: https://fred.stlouisfed.org/series/LFWA64TTUSM647S

 SEX BY AGE [49]
Universe: Total population
Total: P0120001 04 9
Male: P0120002 04 9
Under 5 years P0120003 04 9
5 to 9 years P0120004 04 9
10 to 14 years P0120005 04 9
15 to 17 years P0120006 04 9
18 and 19 years P0120007 04 9
20 years P0120008 04 9
21 years P0120009 04 9
22 to 24 years P0120010 04 9
25 to 29 years P0120011 04 9
30 to 34 years P0120012 04 9
35 to 39 years P0120013 04 9
40 to 44 years P0120014 04 9
45 to 49 years P0120015 04 9
50 to 54 years P0120016 04 9
55 to 59 years P0120017 04 9
60 and 61 years P0120018 04 9
62 to 64 years P0120019 04 9
65 and 66 years P0120020 04 9
67 to 69 years P0120021 04 9
70 to 74 years P0120022 04 9
75 to 79 years P0120023 04 9
80 to 84 years P0120024 04 9
85 years and over P0120025 04 9
Female: P0120026 04 9
Under 5 years P0120027 04 9
5 to 9 years P0120028 04 9
10 to 14 years P0120029 04 9
15 to 17 years P0120030 04 9
18 and 19 years P0120031 04 9
20 years P0120032 04 9
21 years P0120033 04 9
22 to 24 years P0120034 04 9
25 to 29 years P0120035 04 9
30 to 34 years P0120036 04 9
35 to 39 years P0120037 04 9
40 to 44 years P0120038 04 9
45 to 49 years P0120039 04 9
50 to 54 years P0120040 04 9
55 to 59 years P0120041 04 9
60 and 61 years P0120042 04 9
62 to 64 years P0120043 04 9
65 and 66 years P0120044 04 9
67 to 69 years P0120045 04 9
70 to 74 years P0120046 04 9
TABLE (MATRIX) SECTION—Con.
Data Dictionary 6-37
U.S. Census Bureau, 2010 Census Summary File 1
Table
number Table contents
Data
dictionary
reference
name
Segment
Max
size
POPULATION SUBJECTS SUMMARIZED TO THE BLOCK LEVEL—Con.
P12. SEX BY AGE [49]—Con.
Total—Con.
Female—Con.
75 to 79 years P0120047 04 9
80 to 84 years P0120048 04 9
85 years and over P0120049 04 9

In [ ]:
working_age = {
    'P012006': 'Male 15 to 17 years',
    'P012007': 'Male 18 and 19 years',
    'P012008': 'Male 20 years',
    'P012009': 'Male 21 years',
    'P012010': 'Male 22 to 24 years',
    'P012011': 'Male 25 to 29 years',
    'P012012': 'Male 30 to 34 years',
    'P012013': 'Male 35 to 39 years',
    'P012014': 'Male 40 to 44 years',
    'P012015': 'Male 45 to 49 years',
    'P012016': 'Male 50 to 54 years',
    'P012017': 'Male 55 to 59 years',
    'P012018': 'Male 60 and 61 years',
    'P012019': 'Male 62 to 64 years',
    'P012030': 'Female 15 to 17 years',
    'P012031': 'Female 18 and 19 years',
    'P012032': 'Female 20 years',
    'P012033': 'Female 21 years',
    'P012034': 'Female 22 to 24 years',
    'P012035': 'Female 25 to 29 years',
    'P012036': 'Female 30 to 34 years',
    'P012037': 'Female 35 to 39 years',
    'P012038': 'Female 40 to 44 years',
    'P012039': 'Female 45 to 49 years',
    'P012040': 'Female 50 to 54 years',
    'P012041': 'Female 55 to 59 years',
    'P012042': 'Female 60 and 61 years',
    'P012043': 'Female 62 to 64 years'
}

working_age_cols = working_age.keys()
working_age_exp = ' + '.join(working_age_cols)

query = (
    'DROP TABLE IF EXISTS working_age_blocks; '
    'CREATE TABLE working_age_blocks '
    '(geocode, working_age_pop) '
    'WITHOUT OIDS '
    'AS SELECT state || county || tract || block, %s '
    'FROM sf1_04 INNER JOIN geo2010 ON sf1_04.logrecno = geo2010.logrecno AND sf1_04.stusab = geo2010.stusab '
    'WHERE geo2010.sumlev=\'101\' ')% working_age_exp
psql(query, database='census2010')

In [ ]:
psql('ALTER TABLE working_age_blocks '
     'ADD CONSTRAINT geocode_primary PRIMARY KEY (geocode);',
     database='census2010')


ALTER TABLE working_age_blocks ADD CONSTRAINT geocode_primary PRIMARY KEY (geocode);


In [5]:
psql('SELECT SUM(working_age_pop) FROM working_age_blocks;', database='census2010')

SELECT SUM(working_age_pop) FROM working_age_blocks;
Finished execution in 2.01712 secs: sum    
-----------
 209700801
(1 row)


In [6]:
psql('\d geo2010', database='census2010')

\d geo2010
Finished execution in 0.127231 secs: Table "public.geo2010"
  Column   |         Type          | Modifiers 
-----------+-----------------------+-----------
 fileid    | character varying(6)  | not null
 stusab    | character varying(2)  | not null
 sumlev    | character varying(3)  | not null
 geocomp   | character varying(2)  | not null
 chariter  | character varying(3)  | not null
 cifsn     | character varying(2)  | 
 logrecno  | character varying(7)  | not null
 region    | character varying(1)  | 
 division  | character varying(1)  | 
 state     | character varying(2)  | 
 county    | character varying(3)  | 
 countycc  | character varying(2)  | 
 countysc  | character varying(2)  | 
 cousub    | character varying(5)  | 
 cousubcc  | character varying(2)  | 
 cousubsc  | character varying(2)  | 
 place     | character varying(5)  | 
 placecc   | character varying(2)  | 
 placesc   | character varying(2)  | 
 tract     | character varying(6)  | 
 blkgrp    | character va

In [ ]:
!rm working_age_blocks.psql
!pg_dump -t working_age_blocks -f working_age_blocks.psql census2010
!ls -l working_age_blocks.psql

In [16]:
!head -50 working_age_blocks.psql

--
-- PostgreSQL database dump
--

-- Dumped from database version 9.5.4
-- Dumped by pg_dump version 9.5.4

SET statement_timeout = 0;
SET lock_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SET check_function_bodies = false;
SET client_min_messages = warning;
SET row_security = off;

SET search_path = public, pg_catalog;

SET default_tablespace = '';

SET default_with_oids = false;

--
-- Name: working_age_blocks; Type: TABLE; Schema: public; Owner: rsargent
--

CREATE TABLE working_age_blocks (
    geocode text NOT NULL,
    working_age_pop integer
);


ALTER TABLE working_age_blocks OWNER TO rsargent;

--
-- Data for Name: working_age_blocks; Type: TABLE DATA; Schema: public; Owner: rsargent
--

COPY working_age_blocks (geocode, working_age_pop) FROM stdin;
020130001001363	24
010010211001053	0
050014808001218	0
040019442011002	27
060014271001000	0
080010085231000	0
090019900000001	0
110010001001000	263
100010421002025	0
120010002001000	26
130019501

In [12]:
psql('\d+ working_age_blocks', database='census2010')

\d+ working_age_blocks
Finished execution in 0.148423 secs: Table "public.working_age_blocks"
     Column      |  Type   | Modifiers | Storage  | Stats target | Description 
-----------------+---------+-----------+----------+--------------+-------------
 geocode         | text    | not null  | extended |              | 
 working_age_pop | integer |           | plain    |              | 
Indexes:
    "geocode_primary" PRIMARY KEY, btree (geocode)


In [17]:
!psql -d timelapse < working_age_blocks.psql

SET
SET
SET
SET
SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
COPY 11155486
ALTER TABLE


In [18]:
psql('\d+ working_age_blocks')

\d+ working_age_blocks
Finished execution in 0.140417 secs: Table "public.working_age_blocks"
     Column      |  Type   | Modifiers | Storage  | Stats target | Description 
-----------------+---------+-----------+----------+--------------+-------------
 geocode         | text    | not null  | extended |              | 
 working_age_pop | integer |           | plain    |              | 
Indexes:
    "geocode_primary" PRIMARY KEY, btree (geocode)
